In [1]:
# =====================================================
# Cell 1: System Info & Resource Versions
# =====================================================
# This cell shows basic system info—OS version, GPU, default Python,
# pip, conda (if installed) etc. It's useful for debugging or for
# reproducibility info in a shared notebook.

!set -euxo pipefail

print("=== OS Release Info ===")
!lsb_release -a

print("\n=== GPU Info (nvidia-smi) ===")
!nvidia-smi || echo "[WARNING] nvidia-smi not found"

print("\n=== Default nvcc Version (if available) ===")
!nvcc --version || echo "[WARNING] nvcc not found"

print("\n=== Default Python & Pip Version ===")
!which python
!python --version
!pip --version

print("\n=== Conda Version (if any) ===")
!conda --version || echo "[INFO] 'conda' not yet installed."


=== OS Release Info ===
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy

=== GPU Info (nvidia-smi) ===
Sat Jul 19 10:04:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P8             11W /   70W |       0MiB /  153

In [ ]:
# =====================================================
# Cell 2: Install CUDA 11.8
# =====================================================
# Installs CUDA 11.8 from the official NVIDIA repo.
# We pin this version to ensure compatibility with certain libraries.

# Set environment variable for non-interactive installation
import os
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

print("---- Installing CUDA 11.8 ----")

# STEP 0) Update and install required packages
!sudo apt-get update -y
!sudo apt-get install -y gnupg curl

# STEP 1) Preconfigure keyboard settings to avoid prompts
# You can set it to a default layout, e.g., English (US)
!echo "keyboard-configuration keyboard-configuration/layout select English (US)" | sudo debconf-set-selections
!echo "keyboard-configuration keyboard-configuration/modelcode string pc105" | sudo debconf-set-selections
!echo "keyboard-configuration keyboard-configuration/layoutcode string us" | sudo debconf-set-selections

# STEP 2) Download the cuda-keyring .deb from NVIDIA's repo
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb

# STEP 3) Install the cuda-keyring .deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb

# STEP 4) Update apt now that the repo & key are configured
!sudo apt-get update -y

# STEP 5) Install CUDA 11.8 without interactive prompts
!sudo apt-get install -y cuda-11-8

# STEP 6) Set environment variables for CUDA
import sys
import subprocess

# Append CUDA paths to environment variables
cuda_path = "/usr/local/cuda-11.8/bin"
cuda_lib = "/usr/local/cuda-11.8/lib64"

# Update PATH and LD_LIBRARY_PATH
os.environ['PATH'] += f":{cuda_path}"
os.environ['LD_LIBRARY_PATH'] = f"{cuda_lib}:" + os.environ.get('LD_LIBRARY_PATH', '')

# Verify CUDA installation
!nvcc --version


---- Installing CUDA 11.8 ----
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,124 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,932 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get

In [ ]:
# =====================================================
# Cell 3: Double-Check CUDA 11.8 (Optional)
# =====================================================
# In Colab, environment variables may not persist across cells.
# This cell verifies that 'nvcc' is still available and at 11.8.

!export PATH="/usr/local/cuda-11.8/bin:$PATH" && \
nvcc --version


In [ ]:
# =====================================================
# Cell 4: Install Miniconda using condacolab
# =====================================================
# We use `condacolab` to install and configure a Miniconda environment
# inside Colab. Once installed, we still need to "source" and activate
# it in bash cells to use conda commands.

!pip install -q condacolab
import condacolab
condacolab.install_miniconda()


In [ ]:
# =====================================================
# Cell 5: Create & Populate the 'trellis' Environment
# =====================================================
# 1) Source the conda script
# 2) Prepend /usr/local/cuda-11.8 to PATH so that all compilers see the correct CUDA
# 3) Create a fresh conda env named 'trellis' with Python 3.11
# 4) Install PyTorch, flash-attn, spconv, kaolin, etc.

%%bash
set -ex

# Make conda commands available in this shell
source /usr/local/etc/profile.d/conda.sh

# Prepend /usr/local/cuda-11.8 to PATH
export PATH="/usr/local/cuda-11.8/bin:$PATH"
export LD_LIBRARY_PATH="/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH"

# Create & activate the conda environment
conda create -y -n trellis python=3.11
conda activate trellis

# Install PyTorch (2.3.0) + CUDA 11.8 build
conda install -y pytorch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 \
               pytorch-cuda=11.8 -c pytorch -c nvidia

# Install flash-attn 2.5.8 build for PyTorch 2.3 + CUDA 11.8
pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.8/flash_attn-2.5.8+cu118torch2.3cxx11abiFALSE-cp311-cp311-linux_x86_64.whl

# Install spconv for CUDA 11.8
pip install spconv-cu118

# Install kaolin
pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.3.0_cu118.html

# Pillow < 11.0 if needed for certain dependencies
pip install --force-reinstall "pillow<11.0"

# Install Gradio + custom gradio_litmodel3d
pip install gradio==4.44.1 gradio_litmodel3d==0.0.1

echo "[INFO] 'trellis' environment created and packages installed."


In [ ]:
# =====================================================
# Cell 6: System Dev Tools & Python Build Libraries
# =====================================================
# Installs system-level development tools: build-essential, cmake, ninja, nvidia-cuda-toolkit
# Then re-activates the 'trellis' environment and upgrades pip, setuptools, wheel, ninja, Cython.

%%bash
set -ex

apt-get update -y
apt-get install -y build-essential cmake ninja-build nvidia-cuda-toolkit

# Make conda available + activate 'trellis'
source /usr/local/etc/profile.d/conda.sh
conda activate trellis

# Upgrade build libraries in Python
pip install --upgrade pip setuptools wheel ninja Cython

echo "[INFO] System dev tools + Python build libs installed."


May crash at this point. if it does, just resume in the cell below


In [ ]:
# =====================================================
# Cell 7: Clone & Setup Microsoft/TRELLIS
# =====================================================
# 1) Source conda script
# 2) Prepend CUDA 11.8
# 3) Activate trellis
# 4) Clean leftover submodule directories
# 5) Clone the MS/TRELLIS repo
# 6) Create 'assets/example_image' folder so app.py won't crash
# 7) Run setup.sh with all relevant flags
# 8) Patch app.py to launch share=True (so you can share the Gradio link)
# 9) Confirm which nvcc is being used

%%bash
set -ex

# Source conda script
source /usr/local/etc/profile.d/conda.sh

# Prepend /usr/local/cuda-11.8 to PATH
export PATH="/usr/local/cuda-11.8/bin:$PATH"
export LD_LIBRARY_PATH="/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH"

# Activate 'trellis'
conda activate trellis

# Remove leftover submodule directories
rm -rf /tmp/extensions
rm -rf /tmp/pip-req-build-*

# Clone official MS/TRELLIS repo
rm -rf /content/TRELLIS
git clone --recurse-submodules https://github.com/microsoft/TRELLIS.git /content/TRELLIS
cd /content/TRELLIS
git submodule update --init --recursive

# Create missing folder so app.py won't crash
mkdir -p /content/TRELLIS/assets/example_image

# Run setup.sh
bash ./setup.sh --basic --flash-attn --spconv --mipgaussian --nvdiffrast --kaolin --demo || \
  echo "[WARNING] Some submodules likely failed to build, but that's often normal in Colab."

# Patch app.py => use demo.launch(share=True)
sed -i 's/demo.launch()$/demo.launch(share=True)/' app.py

# Confirm which nvcc is being used now
nvcc --version

echo "[INFO] TRELLIS environment set up; 'app.py' patched."


You may see warnings in the out of cell below. safe to ignore.

In [ ]:
# =====================================================
# Cell 8: Clone & Install mip-splatting
# =====================================================
# 1) Clone the 'mip-splatting' repo if not present
# 2) Install required system packages (libgl1-mesa-dev, etc.)
# 3) Prepend CUDA 11.8
# 4) Activate conda
# 5) Install the diff_gaussian_rasterization submodule
#    which is part of mip-splatting

%%bash
set -ex

cd /content
if [ ! -d "/content/mip-splatting" ]; then
  echo "[INFO] 'mip-splatting' folder not found, cloning now..."
  git clone --recurse-submodules https://github.com/autonomousvision/mip-splatting.git
  cd mip-splatting
  git submodule update --init --recursive
  cd ..
else
  echo "[INFO] 'mip-splatting' folder already exists, skipping clone."
fi

apt-get update -y
apt-get install -y libgl1-mesa-dev build-essential cmake ninja-build

# Prepend /usr/local/cuda-11.8 to PATH & LD_LIBRARY_PATH
export PATH="/usr/local/cuda-11.8/bin:$PATH"
export LD_LIBRARY_PATH="/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH"

# Activate 'trellis'
source /usr/local/etc/profile.d/conda.sh
conda activate trellis

# Upgrade Python build tools
pip install --upgrade pip setuptools wheel ninja Cython

# Install diff_gaussian_rasterization from submodule
cd /content/mip-splatting/submodules/diff-gaussian-rasterization
pip install . --no-build-isolation -v

echo "[INFO] diff_gaussian_rasterization installed successfully."


**RUN TRELLIS IN NGROK (PROVIDE YOUR NGROK API KEY)**\
if you run the code below, dont forget to edit and put in your nGrok Auth token\
Also, by default, we load kaiser-app.py \
if you want original app.py, uncomment that line and comment out kaiser-app.py


In [ ]:
# =====================================================
# Cell 9: Launch TRELLIS App with Ngrok (show URL after launch)
# =====================================================
# 0) Download kaiser-app.py from your GitHub (if not present)
# 1) Install/Update ngrok if not present
# 2) Configure ngrok with your token (handle new vs. old versions)
# 3) (Re)install Gradio in 'trellis'
# 4) Start ngrok to expose port 7860
# 5) Launch either 'app.py' or 'kaiser-app.py' in 'trellis' environment
# 6) Print ngrok URL AFTER the Gradio app is actually running

import subprocess, time, json, re, os

# ============== STEP 0) Download kaiser-app.py ==============
# We'll place kaiser-app.py in /content/TRELLIS if it's not already there.
TARGET_DIR = "/content/TRELLIS"
kaiser_app_path = os.path.join(TARGET_DIR, "kaiser-app.py")


print("[INFO] Downloading kaiser-app.py from GitHub into /content/TRELLIS ...")
# Note: We use the raw GitHub link to fetch the raw file contents.
# If you change branches or repos, update this URL accordingly.
KAISER_APP_URL = "https://raw.githubusercontent.com/jackel27/Trellis-Colab/main/kaiser-app.py"
subprocess.run([
    "wget", "-q", "-O", kaiser_app_path, KAISER_APP_URL
], check=True)


# ============== USER CHOICE ==============
# Uncomment exactly ONE of the following lines
# to select which app file you want to run.

#APP_FILE = "app.py" # <-- This is TRELLIS Default app.py
APP_FILE = "kaiser-app.py"  # from https://github.com/jackel27/Trellis-Colab/blob/main/kaiser-app.py

# =========================================


#---------------NGROK AUTH TOKEN-----------
# Replace with your own ngrok auth token
NGROK_AUTH = "305Zvmnll5XKVJF7KC416fUN8cG_5RxKbpVsaGU415p2d1c3k"
#---------------NGROK AUTH TOKEN-----------


# Step 1) Check if ngrok is installed; if not, download & install
try:
    subprocess.run(["which", "ngrok"], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("[INFO] ngrok is already installed.")
except subprocess.CalledProcessError:
    print("[INFO] Installing ngrok ...")
    subprocess.run(["wget", "-q", "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"], check=True)
    subprocess.run(["unzip", "-o", "ngrok-stable-linux-amd64.zip"], check=True)
    subprocess.run(["chmod", "+x", "ngrok"], check=True)
    subprocess.run(["mv", "ngrok", "/usr/local/bin/ngrok"], check=True)

# (Optional) Print ngrok version to see which command we might need
try:
    print("[INFO] Checking ngrok version...")
    subprocess.run(["ngrok", "version"], check=True)
except:
    print("[WARNING] Could not determine ngrok version.")

# Step 2) Configure ngrok (new vs. old command)
try:
    print("[INFO] Configuring ngrok with new command: 'ngrok config add-authtoken' ...")
    subprocess.run(["ngrok", "config", "add-authtoken", NGROK_AUTH], check=True)
except subprocess.CalledProcessError:
    print("[WARNING] 'ngrok config add-authtoken' failed. Trying older command 'ngrok authtoken' ...")
    subprocess.run(["ngrok", "authtoken", NGROK_AUTH], check=True)

# Step 3) (Re)install Gradio in 'trellis'
print("[INFO] Ensuring Gradio is installed in 'trellis' environment...")
subprocess.run([
    "conda", "run", "-n", "trellis",
    "pip", "install", "gradio==4.44.1", "gradio_litmodel3d==0.0.1"
], check=True)

# Step 4) Start ngrok in background, forwarding port 7860
print("[INFO] Starting ngrok on port 7860 ...")
proc_ngrok = subprocess.Popen(["ngrok", "http", "7860"])

# Give ngrok a few seconds to initialize
time.sleep(4)

# Parse ngrok's local API to get the public URL
public_url = None
try:
    tunnels_json = subprocess.check_output(["curl", "-s", "127.0.0.1:4040/api/tunnels"])
    public_url = json.loads(tunnels_json)["tunnels"][0]["public_url"]
except Exception as e:
    print("[ERROR] Could not get ngrok public URL:", e)

# We'll store the URL now, but only print it after the app is ready
if public_url:
    print("[INFO] Found NGROK URL: ", public_url)
    print("[INFO] Ngrok may not be ready yet; you may need to wait until the app is launched.")
else:
    print("[WARNING] No public URL found from ngrok.")

# Step 5) Launch the selected app file in 'trellis' environment, unbuffered
app_path = os.path.join(TARGET_DIR, APP_FILE)  # e.g. /content/TRELLIS/kaiser-app.py
print(f"[INFO] Launching {app_path} in 'trellis' environment ...")

p = subprocess.Popen(
    ["conda", "run", "-n", "trellis", "python", "-u", app_path],
    cwd=TARGET_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True  # so we can read line by line as strings
)

# Step 6) Stream logs. Only print the final NGROK URL after the app is launched
launch_detected = False

try:
    for line in iter(p.stdout.readline, ""):
        if not line:
            break

        # Print the line to the notebook output
        print(line, end="")

        # If we detect "Running on local URL:", we assume the Gradio app is up
        if "Running on local URL:" in line:
            launch_detected = True
            if public_url:
                print(f"\n[INFO] {APP_FILE} is now live!!! Access via ngrok here:\n{public_url}\n")
            else:
                print("\n[WARNING] We have not found an ngrok URL, but the app claims to be running.\n")

except KeyboardInterrupt:
    print("[INFO] Interrupted by user. Stopping the app.")
    p.kill()
finally:
    p.wait()

if not launch_detected and public_url:
    print("[INFO] The app did not print 'Running on local URL:' but here's the NGROK URL anyway:")
    print(public_url)

print("[INFO] Done. If you want to relaunch, re-run this cell.")


**RUN WITH CLOUDFLARE - NO AUTH KEY NEEDED**

In [ ]:
# =====================================================
# Cell 10: Launch TRELLIS App with Cloudflare (show URL after launch)
# =====================================================
# 0) Download kaiser-app.py from your GitHub (if not present)
# 1) Install cloudflared (if not already)
# 2) (Re)install Gradio in 'trellis' environment
# 3) Start a Cloudflare "Quick Tunnel" on localhost:7860
# 4) Launch either 'app.py' or 'kaiser-app.py' in 'trellis' environment
# 5) Print the tunnel URL AFTER the app is actually up.

import subprocess
import time
import re
import os

# ============== STEP 0) Download kaiser-app.py ==============
TARGET_DIR = "/content/TRELLIS"
kaiser_app_path = os.path.join(TARGET_DIR, "kaiser-app.py")

print("[INFO] Downloading kaiser-app.py from GitHub into /content/TRELLIS ...")
KAISER_APP_URL = "https://raw.githubusercontent.com/jackel27/Trellis-Colab/main/kaiser-app.py"
subprocess.run(["wget", "-q", "-O", kaiser_app_path, KAISER_APP_URL], check=True)

# ============== USER CHOICE ==============
# Uncomment exactly ONE of the following lines
# to select which app file you want to run.

#APP_FILE = "app.py"
APP_FILE = "kaiser-app.py"
# =========================================

# Step 1) Install/Update cloudflared via the latest .deb for Linux (amd64)
print("[INFO] Installing/Updating cloudflared...")
try:
    # Download the latest .deb from GitHub
    subprocess.run([
        "wget", "-q", "-O", "cloudflared-linux-amd64.deb",
        "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
    ], check=True)
    # Attempt dpkg install
    subprocess.run(["sudo", "dpkg", "-i", "cloudflared-linux-amd64.deb"], check=True)
except subprocess.CalledProcessError:
    print("[WARNING] cloudflared may already be installed or dpkg encountered an error.")

# Step 2) (Re)install Gradio in 'trellis' environment
print("[INFO] Ensuring Gradio is installed in 'trellis' environment...")
subprocess.run([
    "conda", "run", "-n", "trellis",
    "pip", "install", "gradio==4.44.1", "gradio_litmodel3d==0.0.1"
], check=True)

# Step 3) Start cloudflared in a subprocess to create a Quick Tunnel from :7860
print("[INFO] Starting cloudflared (Quick Tunnel) on port 7860...")
cloudflare_cmd = ["cloudflared", "tunnel", "--url", "http://127.0.0.1:7860"]
proc_cloudflare = subprocess.Popen(
    cloudflare_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Give cloudflared a few seconds to initialize logs
time.sleep(3)

# We'll store the discovered tunnel URL in `tunnel_url`
tunnel_url = None

def extract_cloudflare_url(log_line: str):
    # We look for "https://xxxxx.trycloudflare.com"
    match = re.search(r"https://.*?\.trycloudflare\.com\S*", log_line)
    return match.group(0) if match else None

print("[INFO] Searching cloudflared logs for the public URL...")

# Check up to 10 seconds for a log line containing the tunnel URL
start_time = time.time()
while (time.time() - start_time) < 10:
    if proc_cloudflare.poll() is not None:
        # cloudflared has exited or failed
        break

    line = proc_cloudflare.stdout.readline()
    if not line:
        time.sleep(0.2)
        continue

    possible_url = extract_cloudflare_url(line)
    if possible_url:
        tunnel_url = possible_url
        break

if tunnel_url:
    print(f"[INFO] Found Cloudflare Tunnel URL => {tunnel_url}")
else:
    print("[WARNING] Could not parse the Cloudflare URL from logs yet.")

# Step 4) Launch either app.py or kaiser-app.py in the 'trellis' environment, unbuffered
app_path = os.path.join(TARGET_DIR, APP_FILE)
print(f"[INFO] Launching TRELLIS app ({APP_FILE}) in the 'trellis' environment...")
p_app = subprocess.Popen(
    ["conda", "run", "-n", "trellis", "python", "-u", app_path],
    cwd=TARGET_DIR,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Step 5) Stream logs from the selected app until it prints "Running on local URL:"
launch_detected = False
try:
    for line in iter(p_app.stdout.readline, ""):
        if not line:
            break

        print(line, end="")

        # If the app prints "Running on local URL:", we assume the server is up
        if "Running on local URL:" in line:
            launch_detected = True
            if tunnel_url:
                print(f"\n[INFO] {APP_FILE} is live! Access via Cloudflare:\n{tunnel_url}\n")
            else:
                print("\n[WARNING] We have not found a Cloudflare URL, but the app claims to be running.\n")

except KeyboardInterrupt:
    print("[INFO] Interrupted by user. Stopping the app.")
    p_app.kill()
finally:
    p_app.wait()

# If the app never printed "Running on local URL:", show the URL (if any)
if not launch_detected and tunnel_url:
    print("\n[INFO] The app didn't print 'Running on local URL:' but here's the Cloudflare URL anyway:")
    print(tunnel_url)

print("[INFO] Done. If you'd like to relaunch, just re-run this cell or the entire notebook.")
